In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

Mounted at /content/drive


In [ ]:
pip install kaggle

In [ ]:
!kaggle datasets download -d hyeonwooan/imdb-data-preprocessed-using-ml20m

 90% 66.0M/73.5M [00:00<00:00, 164MB/s]
100% 73.5M/73.5M [00:00<00:00, 147MB/s]


In [ ]:
!unzip /content/imdb-data-preprocessed-using-ml20m.zip

Archive:  /content/imdb-data-preprocessed-using-ml20m.zip
  inflating: actors_vectors.p        
  inflating: actors_vectors_10.p     
  inflating: actors_vectors_100.p    
  inflating: actors_vectors_20.p     
  inflating: actors_vectors_50.p     
  inflating: directors_vectors.p     
  inflating: directors_vectors_10.p  
  inflating: directors_vectors_100.p  
  inflating: directors_vectors_20.p  
  inflating: directors_vectors_50.p  
  inflating: imdb_data.csv           
  inflating: imdb_rating_with_ml20.csv  
  inflating: vocab_actors.p          
  inflating: vocab_directors.p       


In [ ]:
import pandas as pd
metadata=pd.read_csv('imdb_data.csv')
metadata=metadata[1:100]
metadata.columns
metadata.head(1)

,id,name,year,grade,runtime,directors,actors,genre,rating,votes,story,actors_order,directors_order,popularity
1,tt8367814,The Gentlemen,(2019),18,113 min,guy ritchie,"matthew mcconaughey,charlie hunnam,michelle do...","action,comedy,crime",7.9,125948,"expat,marijuana,empire,london,attempt,domain","[96665, 24821, 100619, 66295]",[16615],0.994154


In [ ]:
features = ['name', 'genre', 'actors', 'directors']

In [ ]:
content = metadata[features]

In [ ]:
content=content.dropna()

In [ ]:
content['name'].unique

<bound method Series.unique of 1                                     The Gentlemen
2     Star Wars: Episode IX - The Rise of Skywalker
3                                 Avengers: Endgame
4                                 Bad Boys for Life
5                                            Code 8
                          ...                      
95                                    The Nice Guys
96                                         Bad Boys
97                                  Triple Frontier
98                                    Plagi Breslau
99                         The Fast and the Furious
Name: name, Length: 99, dtype: object>

In [ ]:
content['name']=content['name'].str.lower()
content['genre']=content['genre'].str.lower()
content['actors']=content['actors'].str.lower()
content['directors']=content['directors'].str.lower()

In [ ]:
content['combined']=content['name']+' '+content['genre']+' '+content['actors']+' '+content['directors']

In [ ]:
content.iloc[0]

name                                             the gentlemen
genre                                      action,comedy,crime
actors       matthew mcconaughey,charlie hunnam,michelle do...
directors                                          guy ritchie
combined     the gentlemen action,comedy,crime matthew mcco...
Name: 1, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(stop_words='english')

In [ ]:
tfidf_dict = tfidf.fit_transform(content['combined'])
#tfidf_dict = tfidf.fit_transform(content['combined'])
print(tfidf_dict)
tfidf.vocabulary_

In [ ]:
print(tfidf_dict)

In [ ]:
 from sklearn.metrics.pairwise import linear_kernel

In [ ]:
cosine_sim = linear_kernel(tfidf_dict, tfidf_dict)

In [ ]:
cosine_sim

array([[1.        , 0.00343662, 0.00372035, ..., 0.17447686, 0.0265045 ,
        0.0923904 ],
       [0.00343662, 1.        , 0.01030538, ..., 0.14633028, 0.00314909,
        0.003451  ],
       [0.00372035, 0.01030538, 1.        , ..., 0.01151697, 0.00340908,
        0.00373592],
       ...,
       [0.17447686, 0.14633028, 0.01151697, ..., 1.        , 0.02576444,
        0.02823453],
       [0.0265045 , 0.00314909, 0.00340908, ..., 0.02576444, 1.        ,
        0.05712981],
       [0.0923904 , 0.003451  , 0.00373592, ..., 0.02823453, 0.05712981,
        1.        ]])

In [ ]:
indices = pd.Series(content.index, index=content['name']).drop_duplicates()
idx=indices['the gentlemen']
idx

1

In [ ]:
sim_scores=list(enumerate(cosine_sim[idx]))

In [ ]:
sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)


In [ ]:
sim_scores=sim_scores[1:11]

In [ ]:
movie_indices=[i[0] for i in sim_scores]
content['name'].iloc[movie_indices]

85      star wars: episode viii - the last jedi
51    star wars: episode i - the phantom menace
97                              triple frontier
53                                    star wars
65                      solo: a star wars story
31                                    gladiator
70                                  baby driver
63                      raiders of the lost ark
84                             star trek beyond
88                                 blade runner
Name: name, dtype: object